In [1]:
import pandas as pd
from scipy import linalg
from scipy.stats import f,t
import numpy as np
from collections import Counter
data=pd.read_csv("chickwts.csv")

In [2]:
Y=data['weight'].values
p=len(set(data['feed']))
n=len(Y)
feed_numbers=list(Counter(data['feed']).values())
X=np.zeros([n,p])
for j in range(p):
    for i in range(n):
        if i in range(int(np.sum(feed_numbers[0:j])), int(np.sum(feed_numbers[0:j+1]))):
            X[i][j]=1
beta_hat=linalg.inv(X.T@X)@X.T@Y
sigma_square_hat=linalg.norm(Y-X@beta_hat)**2/n
sigma_square_tilde=sigma_square_hat*n/(n-p)
confidence=0.95
A=X.T@X/(p*sigma_square_tilde*f.ppf(q=confidence, dfn=p, dfd=n-p))

In [3]:
print("The realisation of the random set C(Y) for",confidence,"confidence is the ellipsoid {x: (v-x)'A(v-x)<1}, where")
print("v= \n", beta_hat)
print("A= \n", A)
print("That is, x is in the confidence set if the following is satisified:")
for i in range(p):
    if i!=p-1:
        print(A[i][i],'(',beta_hat[i],'-x_',i+1,')^2+') #this uses that A is diagonal because all the variables are dummy
    else:
        print(A[i][i],'(',beta_hat[i],'-x_',i+1,')^2    <   1')

The realisation of the random set C(Y) for 0.95 confidence is the ellipsoid {x: (v-x)'A(v-x)<1}, where
v= 
 [160.2        218.75       246.42857143 328.91666667 276.90909091
 323.58333333]
A= 
 [[0.00024712 0.         0.         0.         0.         0.        ]
 [0.         0.00029655 0.         0.         0.         0.        ]
 [0.         0.         0.00034597 0.         0.         0.        ]
 [0.         0.         0.         0.00029655 0.         0.        ]
 [0.         0.         0.         0.         0.00027183 0.        ]
 [0.         0.         0.         0.         0.         0.00029655]]
That is, x is in the confidence set if the following is satisified:
0.0002471214125534801 ( 160.20000000000002 -x_ 1 )^2+
0.0002965456950641761 ( 218.74999999999997 -x_ 2 )^2+
0.0003459699775748721 ( 246.42857142857142 -x_ 3 )^2+
0.0002965456950641761 ( 328.91666666666663 -x_ 4 )^2+
0.0002718335538088281 ( 276.90909090909093 -x_ 5 )^2+
0.0002965456950641761 ( 323.5833333333333 -x_ 6 )^2  

In [4]:
individual_confidence=confidence**(1/p)
print("An alternative confidence set, which is a hyperrectangle, is given by those vectors x that satisfy:")
for i in range(p): #this uses that A is diagonal because all the variables are dummy
    print("x_",i+1,"in [",beta_hat[i] - t.ppf(q=individual_confidence, df=n-p)*np.sqrt(sigma_square_tilde)*linalg.sqrtm(linalg.inv(X.T@X))[i][i],",",beta_hat[i] + t.ppf(q=individual_confidence, df=n-p)*np.sqrt(sigma_square_tilde)*linalg.sqrtm(linalg.inv(X.T@X))[i][i],"]")

An alternative confidence set, which is a hyperrectangle, is given by those vectors x that satisfy:
x_ 1 in [ 117.71923926647509 , 202.68076073352495 ]
x_ 2 in [ 179.97054847711445 , 257.5294515228855 ]
x_ 3 in [ 210.5257757505287 , 282.3313671066141 ]
x_ 4 in [ 290.1372151437811 , 367.6961181895522 ]
x_ 5 in [ 236.40527478698175 , 317.4129070312001 ]
x_ 6 in [ 284.8038818104478 , 362.3627848562188 ]


In [5]:
lower_ends=[]
upper_ends=[]
for i in range(p):
    lower_ends.append(beta_hat[i] - t.ppf(q=0.5+confidence/2, df=n-p)*np.sqrt(sigma_square_tilde)*linalg.sqrtm(linalg.inv(X.T@X))[i][i])
    upper_ends.append(beta_hat[i] + t.ppf(q=0.5+confidence/2, df=n-p)*np.sqrt(sigma_square_tilde)*linalg.sqrtm(linalg.inv(X.T@X))[i][i])
print("The individual confidence intervals are:")
for i in range(p):
    print('for Beta_',i+1,": [", lower_ends[i],",",upper_ends[i],"]")

The individual confidence intervals are:
for Beta_ 1 : [ 125.55927500299106 , 194.84072499700898 ]
for Beta_ 2 : [ 187.12748918467517 , 250.37251081532477 ]
for Beta_ 3 : [ 217.1518153104688 , 275.705327546674 ]
for Beta_ 4 : [ 297.2941558513418 , 360.53917748199143 ]
for Beta_ 5 : [ 243.88045556009826 , 309.9377262580836 ]
for Beta_ 6 : [ 291.9608225180085 , 355.2058441486581 ]


In [6]:
Y_bar=np.mean(Y)
print("The intervals for the following parameters exclude the overall mean:")
for i in range(p):
    if (Y_bar < lower_ends[i] or Y_bar > upper_ends[i]):
        print("Beta_",i+1)

The intervals for the following parameters exclude the overall mean:
Beta_ 1
Beta_ 2
Beta_ 4
Beta_ 6
